### Multiling 2019 evaluation

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys

sys.path.append("../")

warnings.filterwarnings("ignore")

In [ ]:
data_path = "/home/lyn/lw/multiling2019/multiling2019_wiki/WikiTrain19/tagged_data/data/"

In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv(data_path + "train.csv", header=-1)
# train_df.to_csv(data_path + "train.csv", index=False, header=['0', '1'])

In [ ]:
valid_df = pd.read_csv(data_path + "valid.csv", header=-1)
# valid_df.to_csv(data_path + "valid.csv", index=False, header=['0', '1'])

In [ ]:
test_df = pd.read_csv(data_path + "test.csv", header=-1)
# test_df.to_csv(data_path + "test.csv", index=False, header=['0', '1'])

In [ ]:
train_df

### 1. Create data loaders

In [ ]:
import os

train_path = data_path + "train.csv"
valid_path = data_path + "valid.csv"
test_path = data_path + "test.csv"

model_dir = " /home/lyn/lw/multiling2019/multiling2019_wiki/ner-bertdatadrive/multi_cased_L-12_H-768_A-12/"
init_checkpoint_pt = os.path.join(
    "/home/lyn/lw/multiling2019/multiling2019_wiki/ner-bert/datadrive/multi_cased_L-12_H-768_A-12/",
    "pytorch_model.bin")
bert_config_file = os.path.join(
    "/home/lyn/lw/multiling2019/multiling2019_wiki/ner-bert/datadrive/multi_cased_L-12_H-768_A-12/",
    "bert_config.json")
vocab_file = os.path.join(
    "/home/lyn/lw/multiling2019/multiling2019_wiki/ner-bert/datadrive/multi_cased_L-12_H-768_A-12/",
    "vocab.txt")

In [ ]:
import torch
torch.cuda.set_device(0)
torch.cuda.is_available(), torch.cuda.current_device()

In [ ]:
from modules import BertNerData as NerData

In [ ]:
data = NerData.create(train_path, valid_path, vocab_file)

In [ ]:
len(data.train_dl.dataset), len(data.valid_dl.dataset)

In [ ]:
print(data.id2label)

In [ ]:
sup_labels = ['B_MISC', 'I_MISC']

In [ ]:
max([len(f.labels_ids) for f in data.train_dl.dataset])

### 2. Create model

In [ ]:
from modules.models.bert_models import BertBiLSTMAttnCRF

In [ ]:
model = BertBiLSTMAttnCRF.create(len(data.label2idx),
                                 bert_config_file,
                                 init_checkpoint_pt,
                                 enc_hidden_dim=256)

In [ ]:
model.get_n_trainable_params()

#### TODO: fix bug with len

### 3. Create Learner

In [ ]:
from modules import NerLearner

In [ ]:
num_epochs = 10
learner = NerLearner(
    model,
    data,
    best_model_path=
    "/home/lyn/lw/multiling2019/multiling2019_wiki/ner-bert/datadrive/models/multiling-2019/BertBiLSTMAttnCRF.cpt",
    lr=0.001,
    clip=1.0,
    sup_labels=sup_labels,
    t_total=num_epochs * len(data.train_dl))

### 4. Start learning

In [ ]:
learner.fit(num_epochs, target_metric='f1')

### 5. Evaluate dev set

In [ ]:
from modules.data.bert_data import get_bert_data_loader_for_predict
dl,f = get_bert_data_loader_for_predict(data_path + "valid.csv", learner)

In [ ]:
learner.load_model()

In [ ]:
preds = learner.predict(dl)

In [ ]:
preds

# IOB precision

In [ ]:
from modules.train.train import validate_step
print(
    validate_step(learner.data.valid_dl, learner.model, learner.data.id2label,
                  learner.sup_labels))

# Span precision

In [ ]:
from modules.utils.plot_metrics import get_bert_span_report
clf_report = get_bert_span_report(dl, preds, [])
print(clf_report)

### 6. Evaluate test set

In [ ]:
from modules.data.bert_data import get_bert_data_loader_for_predict
dl,f = get_bert_data_loader_for_predict(data_path + "test.csv", learner)

In [ ]:
preds = learner.predict(dl)

In [ ]:
data = NerData.create(train_path, data_path + "test.csv", vocab_file)

# IOB precision

In [ ]:
from modules.train.train import validate_step
print(
    validate_step(data.valid_dl, learner.model, learner.data.id2label,
                  learner.sup_labels))

# Span precision

In [ ]:
from modules.utils.plot_metrics import get_bert_span_report
clf_report = get_bert_span_report(dl, preds, [])
print(clf_report)

In [ ]:
import os

In [ ]:
total = 0
for lang in os.listdir("/home/lyn/lw/multiling2019/multiling2019_wiki/WikiTrain19/full/"):
    count = 0
    for _ in os.listdir("/home/lyn/lw/multiling2019/multiling2019_wiki/WikiTrain19/full/"+lang):
        count += 1
        total += 1
    print(lang,count)
print(total)